In [1]:
import pandas
pbp=pandas.read_csv('./film/hun1718.csv')

In [2]:
boxscore=pandas.read_csv('./boxscore/gamehunx1718.csv',header=None)
boxscore.columns=["CODE","team","GM","Min","PF","DF","2Pm","2Pa","2PP","3Pm","3Pa","3PP","FTm","Fta","FTP","ORB","DRB","TRB","BS","SR","TOV","ST","ASS","PTS", "VALU","OP","OER","ASTO","VIR","extra1","date","extra3","nat"]

In [67]:
boxscore=boxscore[boxscore['team']==9002]
boxscore=boxscore[boxscore['GM']==1]
playedgames=boxscore['extra1'].unique()

In [71]:
starters=dict()
cnt=0
for game in playedgames:
    starter=[]
    for i,row in boxscore.iterrows():
        if((row['team']==9002)&(row['extra1']==game)):
            starter.append(row['CODE'])
    tmp=frozenset(starter)
    if(tmp not in starters):
        starters.update({tmp:cnt})
        cnt+=1
starters

{frozenset({'A32321', 'A45500', 'A52566', 'A53266', 'A65338'}): 0,
 frozenset({'A30765', 'A32321', 'A45500', 'A52566', 'A53266'}): 1,
 frozenset({'A45500', 'A52566', 'A53266', 'A55019', 'A65338'}): 2,
 frozenset({'A36491', 'A45500', 'A52566', 'A53266', 'A65338'}): 3,
 frozenset({'A30765', 'A45500', 'A52566', 'A55019', 'A81848'}): 4,
 frozenset({'A30765', 'A45500', 'A52566', 'A53266', 'A55019'}): 5}

In [5]:
#Govens playerid-je A52566

In [76]:
pbp['gamecode']=pbp['gamecode'].str.replace('hun_','').astype('int64')
pbp

,id,year,lea,gamecode,teamid,side,playername,playername2,playercode,playercode2,...,subcode,rel_id,period,minute,sec,x,y,area,original_text,sorrend
0,96989122,x1718,hun,927,9213,0,SARLIJA Marino,NaN,A33059,NaN,...,0,0,2,1,0,0,0,NaN,NaN,130
1,97168871,x1718,hun,853,9218,1,MISEK Levente,NaN,A41892,NaN,...,0,0,3,6,0,0,0,NaN,NaN,290
2,96989329,x1718,hun,927,9213,0,BUDIMIR Veljko,NaN,A28736,NaN,...,0,0,2,6,0,0,0,NaN,NaN,177
3,96989032,x1718,hun,927,9230,1,CSEH Gyorgy,NaN,A33932,NaN,...,2,0,1,10,0,136,84,NaN,NaN,112
4,96989365,x1718,hun,927,9213,0,SUPOLA Zoltan,NaN,A35371,NaN,...,0,0,2,7,0,0,0,NaN,NaN,186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85123,97200297,x1718,hun,800,40000,1,JONES Jahmal,NaN,A78008,NaN,...,1,0,4,10,0,135,34,NaN,NaN,485
85124,97200298,x1718,hun,800,9013,0,BOLTIC Marko,NaN,A32360,NaN,...,0,0,4,10,0,0,0,NaN,NaN,486
85125,97200299,x1718,hun,800,0,0,NaN,NaN,NaN,NaN,...,0,0,4,10,0,0,0,NaN,NaN,487
85126,97200300,x1718,hun,800,0,0,NaN,NaN,NaN,NaN,...,0,0,5,0,0,0,0,NaN,NaN,488


In [65]:
results=pandas.read_csv('./Meccsek eredményei/result1718.csv')

In [68]:
szoros=results[results.gameid.isin(playedgames)]
szoros.shape

(26, 5)

In [69]:
games=[]
for i,row in szoros.iterrows():
    if(abs(row['PTS1']-row['PTS2'])<8):
        print(row['gameid'])
        games.append(row['gameid'])

860
868
903
925
928
939
953


In [30]:
szoros=szoros[szoros.gameid.isin(games)]

In [70]:
playedgames=games
playedgames

[860, 868, 903, 925, 928, 939, 953]

In [77]:
hashtable=dict()
cnt=0
for game in playedgames:
    starter=set()
    for i,row in boxscore.iterrows():
        if((row['team']==9002)&(row['extra1']==game)):
            starter.add(row['CODE'])
    tmp=frozenset(starter)
    if(tmp not in hashtable):
        hashtable.update({tmp:cnt})
        cnt+=1
    pbp_game=pbp[pbp['gamecode']==game].sort_values(by=['sorrend'])
    for i,row in pbp_game.iterrows():
         if((row['teamid']==9002)&(row['fccode']==1011)):
            starter.add(row['playercode'])
            starter.discard(row['playercode2'])
            tmp=frozenset(starter)
            if(tmp not in hashtable):
                hashtable.update({tmp:cnt})
                cnt+=1   

In [78]:
inorder_hash=[]
played_mins=[]
for game in playedgames:
    lineup=set()
    for i,row in boxscore.iterrows():
        if((row['team']==9002)&(row['extra1']==game)):
            lineup.add(row['CODE'])
    tmp=frozenset(lineup)
    inorder_hash.append(hashtable.get(tmp))
    fr=0
    pbp_game=pbp[pbp['gamecode']==game].sort_values(by=['sorrend'])
    for i,row in pbp_game.iterrows():
         if((row['teamid']==9002)&(row['fccode']==1011)):
            to=int((row['period']-1)*10+row['minute'])
            played_mins.append(to-fr)
            lineup.add(row['playercode'])
            lineup.discard(row['playercode2'])
            tmp=frozenset(lineup)
            inorder_hash.append(hashtable.get(tmp))
            fr=to
    played_mins.append(40-fr)

In [50]:
def get_points(row):
    if((row['fccode']==1000)&(row['teamid']==9002)):
            if(row['subcode']==3):
                return 3
            elif(row['subcode']==4):
                return 1
            else:
                return 2
    return 0

In [51]:
def get_otherMetrics(row,param,param2=-1):
    if((row['fccode']==param)|(row['fccode']==param2)):
        return 1
    return 0

In [79]:
metrics=[]
for game in playedgames:
    points=0
    shoots=0
    missed=0
    defrb=0
    offrb=0
    steal=0
    turnover=0
    assist=0
    pbp_game=pbp[pbp['gamecode']==game].sort_values(by=['sorrend'])
    for i,row in pbp_game.iterrows():
        if(row['teamid']==9002):
            points+=get_points(row)
            shoots+=get_otherMetrics(row,1000,1001)
            missed+=get_otherMetrics(row,1001)
            defrb+=get_otherMetrics(row,1002,2002)
            offrb+=get_otherMetrics(row,1003,2003)
            steal+=get_otherMetrics(row,1004,2004)
            turnover+=get_otherMetrics(row,1005,2005)
            assist+=get_otherMetrics(row,1008)
            if(row['fccode']==1011):
                metrics.append([points,shoots,missed,defrb,offrb,steal,turnover,assist])
                points=0
                shoots=0
                missed=0
                defrb=0
                offrb=0
                steal=0
                turnover=0
                assist=0
    metrics.append([points,shoots,missed,defrb,offrb,steal,turnover,assist])  

In [80]:
df=pandas.DataFrame(metrics)
df.columns=['points','shots','missed','defrb','offrb','steal','turnover','assist']
df['min']=played_mins
df['hash']=inorder_hash
df.groupby('hash').sum().sort_values(by=['min','points'],ascending=False).head(20)

,points,shots,missed,defrb,offrb,steal,turnover,assist,min
hash,,,,,,,,,
1,124,125,60,44,17,9,20,32,66
19,89,81,31,23,10,8,9,20,41
10,34,46,26,17,5,3,7,6,22
4,48,47,22,14,6,2,3,9,20
5,42,39,16,9,5,5,2,9,20
2,36,32,12,10,2,3,2,8,16
9,31,30,13,9,2,1,5,7,11
13,28,26,10,5,4,2,2,4,11
28,22,30,14,8,2,2,0,4,10


In [81]:
isin=[]
for lineup in hashtable:
    if('A52566' in lineup):
        isin.append(hashtable.get(lineup))
len(isin)

26

In [82]:
df.groupby(df['hash'].isin(isin)).sum()

,points,shots,missed,defrb,offrb,steal,turnover,assist,min,hash
hash,,,,,,,,,,
False,23,32,19,8,8,2,2,2,13,125
True,639,639,287,196,84,47,64,141,267,724


In [83]:
result=df.groupby(df['hash'].isin(isin)).sum()
for i,row in result.iterrows():
    for j in range(8):
        row[j]=row[j]/row['min']*40
result

,points,shots,missed,defrb,offrb,steal,turnover,assist,min,hash
hash,,,,,,,,,,
False,70,98,58,24,24,6,6,6,13,125
True,95,95,42,29,12,7,9,21,267,724
